<a href="https://colab.research.google.com/github/Blockchain2219/RainFallPrediction/blob/main/RainFall_Prediction_Using_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import os
import pandas as pd
import datetime
import serial
import time
import requests

from google.colab import output
from pyngrok import ngrok
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.regularizers import l2
from time import time
from tensorflow.python.framework import dtypes
from tensorflow_io.bigquery import BigQueryClient
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, BatchNormalization, Dense, Conv1D, MaxPooling1D, Flatten, SimpleRNN

In [ ]:
!pip install numpy
!pip install pandas
!pip install pyngrok
!pip install scikit-learn
!pip install keras
!pip install tensorflow
!pip install tensorflow-io
!pip install serial
!pip install pyserial
!apt-get install socat
!pip install paho-mqtt

Configuring Serial Port and Baud RATE OF NODEMCU

Code to Send Data to NodeMCU

In [ ]:
api_key = "CCX3GWFFS2ATDV8K"
temp=0
humidity=0
predRainfall=0

In [ ]:
def send_data(temp,humidity,predRainFall):
  url = f"https://api.thingspeak.com/update?api_key={api_key}&field1={humidity}&field2={temp}&field3={predRainFall}"
  # Send HTTP GET request to update the channel
  response = requests.get(url)
  # Check the response status code
  if response.status_code == 200:
    print("Data sent to ThingSpeak successfully.")
  else:
    print("Error sending data to ThingSpeak.")


Rain Prediction(Deep Learning)

Load and preprocess the data

In [ ]:
df = pd.read_csv("cleaned_weatherAUS.csv")
Direction = ['N', 'NNE', 'NE', 'ENE', 'E', 'ESE', 'SE', 'SSE', 'S', 'SSW', 'SW', 'WSW', 'W', 'WNW', 'NW', 'NNW']
Degree = []

x = 0
for i in range(0, 337, 22):
    Degree.append(i + x)
    x = x + 0.5

Direction_Degree = dict(zip(Direction, Degree))

df = df.replace({"WindGustDir": Direction_Degree})
df = df.replace({"WindDir9am": Direction_Degree})
df = df.replace({"WindDir3pm": Direction_Degree})

df = df.drop(columns=['Date', 'Location', 'RainTomorrow','Rainfall'], axis=1)
cat_f = [x for x in df.columns if df[x].dtype == 'object']

for name in cat_f:
    enc = LabelEncoder()
    enc.fit(list(df[name].values.astype('str')) + list(df[name].values.astype('str')))
    df[name] = enc.transform(df[name].values.astype('str'))

X = df.drop(['RainProbability'], axis=1)
Y = df['RainProbability']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.1)
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [ ]:
# Reshape the data for LSTM
x_train_lstm = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_test_lstm = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

# Create the LSTM model
model = Sequential()
model.add(LSTM(64, input_shape=(x_train.shape[1], 1), return_sequences=True))
model.add(BatchNormalization())
model.add(LSTM(64))
model.add(BatchNormalization())
model.add(Dense(1))

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='mse')

In [ ]:
# Set up callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

In [ ]:
# Train the model
history = model.fit(x_train_lstm, y_train, epochs=1000 ,validation_split=0.6, callbacks=[reduce_lr, early_stopping])

In [ ]:
!pip install tensorflow
!pip install keras
!pip install matplotlib

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.utils import plot_model
# Visualize the model
plot_model(model, to_file='model.png', show_shapes=True)

In [ ]:
model.summary()

In [ ]:
# Make predictions
y_pred = model.predict(x_test_lstm)

In [ ]:
# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)
print("R2 Score:", r2)

In [ ]:
# Save the model
model.save("lstm_model")

In [ ]:
def PredictRainfall(humidity, temp):
  x_test_selected = x_test[[humidity, temp]]
  num_features = x_train.shape[1]
  x_test_selected = np.reshape(x_test_selected, (x_test_selected.shape[0], num_features, 1))
  y_pred_selected = model.predict(x_test_selected)
  predicted_rainfall = y_pred_selected[1] * 100

  # Extract scalar value from NumPy array
  predicted_rainfall_scalar = predicted_rainfall.item()

  # Convert negative value to positive
  predicted_rainfall_abs = abs(predicted_rainfall_scalar)

  # Format the output as a string with 2 decimal places
  predicted_rainfall_message = "The predicted rainfall is {:.2f}%".format(predicted_rainfall_abs)
  sendMessageToWhatsApp(predicted_rainfall_message)
  send_data(temp, humidity, predicted_rainfall_abs)


In [ ]:
PredictRainfall(234,215) #Humidity,Temp

In [ ]:
import requests
import urllib.parse

phoneNumber = "PHONE NUMBER"
apiKey = "API KEY FOR THAT NUMBER"

def sendMessageToWhatsApp(message):
    url = "http://api.callmebot.com/whatsapp.php"
    payload = {
        'phone': phoneNumber,
        'apikey': apiKey,
        'text': message
    }

    encoded_payload = urllib.parse.urlencode(payload)
    full_url = url + '?' + encoded_payload

    headers = {
        'Content-Type': 'application/x-www-form-urlencoded'
    }

    response = requests.post(full_url, headers=headers)

    if response.status_code == 200:
        print("Message sent successfully")
    else:
        print("Error sending the message")
        print("HTTP response code:", response.status_code)


Main Function For Everything


In [ ]:
%%writefile tracker.txt
110

In [ ]:
import pandas as pd
import time
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Specify the file path of the CSV file in your Google Drive
file_path = '/content/drive/MyDrive/IOTProject/ML.csv'

# Check the last accessed row from the tracker file
tracker_file = 'tracker.txt'
current_row = 0

try:
    with open(tracker_file, 'r') as file:
        current_row = int(file.read())
except FileNotFoundError:
    pass

while True:
    # Read the CSV file
    df = pd.read_csv(file_path, skiprows=current_row, nrows=1)
    #print(df.columns)  # Check the column names
    # Perform operations with the first row data
    # For example, printing the data
    humidity = int(df.iloc[0][1])  # Access humidity from column 1 and convert to float
    temperature = int(df.iloc[0][0])  # Access temperature from column 0 and convert to float
    #print(humidity,temp)
    PredictRainfall(humidity, temperature)
    # Update the tracker file
    with open(tracker_file, 'w') as file:
        file.write(str(current_row + 2))

    # Wait for some time before reading the next row
    # You can adjust the delay time as per your requirement
    time.sleep(5)  # Delay of 5 seconds before reading the next row
